In [178]:
import pandas as pd

In [179]:
from nltk.tokenize import word_tokenize 

def format_sentence(sent):
    return {word: True for word in word_tokenize(sent)}

format_sentence('this is a good boy')

{'this': True, 'is': True, 'a': True, 'good': True, 'boy': True}

In [180]:
data = pd.read_csv('Twitter_Data.csv',dtype=str).values.tolist()

# Cleaning & Preparing the data to train

In [181]:
import re

pos_data = []
neg_data = []
neu_data = []

for rew in range(len(data)):
    record = data[rew]
    line = str(record[0])
    line = line.lower()
    line = re.sub('\s+', ' ', line) #exstra space
    line = re.sub("[^0-9A-Za-z ]", "", line) #Punctuations 
    category = float(record[1])
    if(category > 0):
        pos_data.append([format_sentence(line), 'pos'])
    elif(category < 0):
        neg_data.append([format_sentence(line), 'neg'])
    elif(category == 0):
        neu_data.append([format_sentence(line), 'neu'])
    

In [182]:
training_data = pos_data[:70000] + neg_data[:34000] + neu_data[:53500]
testing_data  = pos_data[70000:] + neg_data[34000:] + neu_data[53500:]

In [183]:
sum = len(pos_data) + len(neg_data) + len(neu_data)
print(len(training_data), len(testing_data), sum)
print(len(pos_data), len(neg_data), len(neu_data))

157500 5473 162973
72250 35510 55213


In [184]:
from nltk.classify import NaiveBayesClassifier

In [185]:
model = NaiveBayesClassifier.train(training_data)

In [186]:
print(model.classify(format_sentence('this is a very nice boy!')))
print(model.classify(format_sentence('surat women perform yagna seeks divine grace for narendra modi become again!')))
print(model.classify(format_sentence('how much clock?')))

pos
neu
pos


In [187]:
from nltk.classify.util import accuracy
accuracy(model, testing_data)

0.5558194774346793